# 1.1 웹페이지 크롤링
---

- 웹페이지 추출시 HTTP 헤더와 HTML의 meta 태그를 기반으로 인코딩 방식을 판별 필요
- 표준 라이브러리 urlib.request 모듈을 사용하여 웹페이지 추출
- urlib.request에 포함돼 있는 urlopen() 함수에 URL을 지정하면 웹페이지 추출가능
- HTTP 헤더를 변경 불가, Basic 인증 사용을 위한 복잡한 처리 필요
- HTTP 헤더 변경 및 Basic 인증을 위해 urlib 대신 Requests 모듈 사용 필요

#### 2.4.1.1 urllib 실습

In [1]:
from urllib.request import urlopen

In [2]:
f = urlopen('http://hanbit.co.kr')

In [4]:
# urlopen() 함수는 HTTPResponse 자료형의 객체를 반환
# 파일 객체이므로 open() 함수로 반환되는 파일객체처럼 핸들링
type(f)

http.client.HTTPResponse

In [9]:
# read() 메서드로 HTTP 응답 본문을 추출
# HTTP 연결은 자동으로 close되므로 별도의 close 함수 X
# f.read()

In [7]:
f.status

200

In [8]:
f.getheader('Content-Type')

'text/html; charset=UTF-8'

- HTTPRespone.read() 메서드로 추출할 수 있는 HTTPResponse 본문은 bytes 자료형

- 문자열로 다루려면 문자 코드를 지정해서 디코딩 필요
- 최근에는 HTML5의 기본 인코딩 방식인 UTF-8 전제로 디코딩 가능
- 한국어 사이트를 크롤링 시 여러가지 인코딩이 혼합되어 있을 수 있으므로 HTTP 헤더를 참조해서 적절한 인코딩 방식으로 디코딩 필요

---

## HTTP 헤더에서 인코딩 방식 추출

- HTTP 응답의 Content-Type 헤더를 참조하면 해당 페이지의 인코딩 방식을 확인 가능

- 한국어가 포함된 페이지의 일반적인 Content-Type 헤더
    - text/html
    - text/html; charset = UTF-8
    - text/html; charset = EUC-KR
    
- UTF-8과 EUC-KR이 해당 페이지의 인코딩 방식
- 인코딩이 명시돼 있지 않은 경우 UTF-8로 간주
- HTTPMessage 객체의 get_content_charset() 메서드를 사용하여 인코딩 추출가능

In [10]:
import sys
from urllib.request import urlopen

In [11]:
f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')

In [12]:
# HTTP 헤더를 기반으로 인코딩 방식을 추출
encoding = f.info(). get_content_charset(failobj="utf-8")

In [13]:
# 인코딩 방식을 표준 오류에 출력
print('encoding:', encoding)

encoding: utf-8


In [14]:
# 추출한 인코딩 방식으로 디코딩
text = f.read().decode(encoding)

In [16]:
# 웹 페이지의 내용을 표준 출력에 출력
# print(text)

---

## meta 태그에서 인코딩 방식 추출


- 웹 서버 설정에 따라 HTTP 헤덩의 Content-Type 인코딩과 실제 사용되고 있는 인코딩 형식 상이
- 브라우저는 HTML 내부의 meta 태그 또는 응답 본분의 바이트열 확인을 통해 최종 인코딩 방식 결정 및 화면 출력
- 디코딩 처리에서 UnicodeDecodeError 발생 시 이러한 방식으로 구현 가능
- HTML meta에 명시되는 인코딩 형식
    - "<meta charset='utf-8'>"
    - "<meta ... ; charset='EUC_KR">"

In [28]:
import re
import sys
from urllib.request import urlopen

In [29]:
f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')
# bytes 자료형의 응답 본문을 일단 변수에 저장
bytes_content=f.read()

In [30]:
# charset은 HTML의 앞부분에 적혀 있는 경우가 많으므로
# 응답 본문의 앞부분 1024바이트를 ASCII 문자로 디코딩해줍니다.
scanned_text = bytes_content[:1024].decode('ascii', errors='replace')

In [33]:
# 디코딩한 문자열에서 정규 표현식으로 charset 값을 추출
match = re.search(r'charset=["\']?([\w-]+)', scanned_text)

In [34]:
match.group(1)

'utf-8'

In [35]:
if match:
    encoding=match.group(1)
else:
    # charset이 명시돼 있지 않으면 UTF-8을 사용
    encoding='utf-8'

In [36]:
# 추출한 인코딩을 표준 오류에 출력
print('encoding:',encoding,file=sys.stderr)

encoding: utf-8


In [38]:
# 추출한 인코딩으로 다시 디코딩
text=bytes_content.decode(encoding)
# 응답 본문을 표준 출력에 출려
# print(text)